<a href="https://colab.research.google.com/github/DanielOlson/CompBioAsia/blob/main/CompBioAsia_Neurons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neruons and small networks
In this notebook we're going to be playing with small collections of neurons using hand picked weights. Our big goal here is to gain an intution of the sorts of things that are possible with neural networks.

## Setup:
There's nothing that you need to do in this section except run each cell once. Running these cells installs and imports needed libraries and gives us foundational classes for our later code. Don't worry about understanding (or even reading) this section - important parts will be explained later!

In [ ]:
# To run cells, just press the play button to the left
# This cell will import some basic tools that we will find useful later!

!pip3 install graphviz
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import graphviz
from IPython.display import Image, display
from math import tanh

In [ ]:
# We include 3 helper classes:
#   Neuron (which has a collection of weights, a bias, and an activation)
#   NeuronLayer (a collection of neurons)
#   NeuralNetwork (a collection of layers)
#
# NeuralNetwork also contains functions to draw a graph of the network

class Neuron(nn.Module):
  def __init__(self, 
               num_weights, 
               activation=torch.tanh):
    
    super(Neuron, self).__init__()
    self.w = 2 * (torch.rand(num_weights, requires_grad=False) - 0.5)
    self.b = 2 * (torch.rand(1, requires_grad=False) - 0.5)
    self.activation = activation
    self.value = 0

  def forward(self, x, train=False):
    self.value = float(self.activation(torch.dot(x, self.w) + self.b).detach())
    if train:
      return self.activation(torch.dot(x, self.w) + self.b)
    else:
      with torch.no_grad():
        return self.activation(torch.dot(x, self.w) + self.b)

    

class NeuronLayer(nn.Module):
  def __init__(self, num_neurons, 
               weights_per_neuron, 
               activation=torch.tanh):
    super(NeuronLayer, self).__init__()
    self.neurons = [Neuron(weights_per_neuron, activation=activation) for _ in range(num_neurons)]
  
  def forward(self, x, train=False):
    output = torch.zeros(len(self.neurons), requires_grad=train)
    for i, neuron in enumerate(self.neurons):
      output[i] = neuron(x)
    return output

def value_to_color(value):
  if value > 1:
    value = 1

  elif value < -1:
    value = -1
  
  
  if value == 0:
    return "#{:2X}{:2X}{:2X}')".format(55, 55, 55)

  elif value > 0:
    value = int(225 * value) + 30
    return "#{:2X}{:2X}{:2X}')".format(55, 55, value)
  
  else:
    value = int(-225 * value) + 30
    return "#{:2X}{:2X}{:2X}')".format(value, 55, 55)
  #str(red) + " 1.0 " + str(blue)


class NeuralNetwork(nn.Module):
  def __init__(self, num_inputs, layers, activation=torch.tanh):
    super(NeuralNetwork, self).__init__()
    self.num_inputs = num_inputs
    
    self.layers = []
    for layer in layers:
      self.layers.append(NeuronLayer(layer, num_inputs, activation=activation))
      num_inputs = layer
    self.layers = nn.ModuleList(self.layers)
    

  def draw_net(self, label_edges = False):
    dot = graphviz.Digraph('tmp', format='png')
    dot.graph_attr['rankdir'] = 'LR'
    dot.graph_attr['dpi'] = '100'
   # dot.graph_attr['rank'] = 'same'
    edge_num = 0
    for i in range(self.num_inputs):
      dot.node('-1,'+str(i), 'i,'+str(i), shape='box')


    #gross
    prev = self.num_inputs
    
    for i, layer in enumerate(self.layers):
      for j, n in enumerate(layer.neurons):
        dot.node(str(i)+','+str(j), str(i)+','+str(j))
        for w in range(prev):
          edge_kwarg = dict()
          if label_edges:
            edge_kwarg['label']=str(i + 1) + "," + str(j + 1) + "," + str(w + 1)
          if w == int(((prev) / 2.0)):
            dot.edge(str(i-1)+','+str(w), str(i)+','+str(j), **edge_kwarg)
          else:
            dot.edge(str(i-1)+','+str(w), str(i)+','+str(j), constraint='false', **edge_kwarg)
              
      prev = len(layer.neurons)
    dot.render(view=True)
    display(Image('tmp.gv.png'))

  def forward(self, x, draw=False, train=False):
    if draw:
      dot = graphviz.Digraph('tmp', format='png')
      dot.graph_attr['rankdir'] = 'LR'
      dot.graph_attr['dpi'] = '60'
      for i in range(self.num_inputs):
        dot.node('-1,'+str(i), 'i,'+str(i), shape='box', fontcolor='white',
                 style='filled', color = value_to_color(x[i]))
    
    for layer in self.layers:
      x = layer(x, train=train)

    if draw:
      prev = self.num_inputs
      for i, layer in enumerate(self.layers):
        for j, n in enumerate(layer.neurons):
          dot.node(str(i)+','+str(j), str(i)+','+str(j), fontcolor='white',
                 style='filled', color = value_to_color(n.value))
          for w in range(prev):
            if w == int(((prev) / 2.0)):
              dot.edge(str(i-1)+','+str(w), str(i)+','+str(j))
            else:
              dot.edge(str(i-1)+','+str(w), str(i)+','+str(j), constraint='false')
              
        prev = len(layer.neurons)
      dot.render(view=True)
      display(Image('tmp.gv.png'))

    return x

# Tasks:


# comparison task
def generate_comparison_data():
  return torch.rand(4)
def test_comparison_result(input, output):
  return ((input[0] + input[1]) - (input[2] + input[3])) * output > 0, ((input[0] + input[1]) - (input[2] + input[3]))


# and task
def generate_AND_data():
  x = torch.randint(0, 2, (2,), dtype=torch.float32)
  x[x == 0] = -1.0
  return x
def test_AND_result(input, output):
  if input[0] == 1 and input[1] == 1:
    return output > 0, 1
  else:
    return output < 0, -1

# or task
def generate_OR_data():
  x = torch.randint(0, 2, (2,), dtype=torch.float32)
  x[x == 0] = -1.0
  return x
def test_OR_result(input, output):
  if input[0] == 1 or input[1] == 1:
    return output > 0, 1
  else:
    return output < 0, -1

# xor task
def generate_XOR_data():
  x = torch.randint(0, 2, (2,), dtype=torch.float32)
  x[x == 0] = -1.0
  return x
def test_XOR_result(input, output):
  if input[0] * input[1] < 0:
    return output > 0, 1
  else:
    return output < 0, -1

def test_network(network, generator, tester):
  for i in range(100):
    input = generator()
    out = network(input)
    result, expected = tester(input, out)
    if not result:
      print("Failed with input", input)
      print("Received output", out)
      network(input, draw=True)
      return
  print("Success!")


## Simple Neural Networks
In the above code I have created a neural network class that allows us to create and edit simple networks. In the cells below we'll take a look at some of the features of that class.

In [ ]:
# To create a simple neural network we need to know two things:
#   1. How many input features there are
#   2. How many layers there will be and how large those layers are.
# 
# Here's an example of how to create a neural network that expects:
#   2 input dims, and 2 layers (layer1: 3 dims, layer2: 1 dim)

network = NeuralNetwork(2, [3, 1])

# We can then visualize our network by using the .draw_net() method.
# Box shaped nodes represent inputs, ovals represent neurons

network.draw_net()


**Task 1**


---


Your first task is to create a network with 4 inputs and 3 layers (of sizes 3, 2, 1 respectively). I've put '???' where you should write some code.

In [ ]:
network = NeuralNetwork(???, ???)
network.draw_net()


In [ ]:
# Initially all the weights and bias for our networks are random values between -1 and 1
#
# To demonstrate that we're going to pass the value '1' through
#   random initializations of a network and see what happens

input = torch.ones(1)

for _ in range(3):
  network = NeuralNetwork(1, [3, 5, 7, 1])
  print(network(input))

# Even though the input doesn't change in the loop, 
#   each itteration has a different output because
#   each itteration has a network with different weights/biases


In [ ]:
# We can visaulize the activation of our neurons by adding
#   'draw=True' inside of the call to our network. 
# Positive activations are blue, and negative activations are red.
# The brighter the color, the more activated the neuron is.

input = torch.ones(1)

for _ in range(3):
  network = NeuralNetwork(1, [3, 5, 7, 1])
  print(network(input, draw=True))
  

In [ ]:
# Lets create an input of all ones and pass that into 
# a network that has only 1 neuron

input = torch.ones(3)
network = NeuralNetwork(3, [1])
print("Random weights:")
out = network(input, draw=True)

# If we wanted all the weights to be zero then we need to 
# set each weight for each neuron in each layer to 0
# Our network only has 1 layer with 1 neuron and 3 weights, so its easy!

network.layers[0].neurons[0].w[0] = 0 # set the 0th weight
network.layers[0].neurons[0].w[1] = 0 # set the 1st weight
network.layers[0].neurons[0].w[2] = 0 # set the 2nd weight
network.layers[0].neurons[0].b = 0    # set the bias to 0

print("Zero weights:")
out = network(input, draw=True)

# We can also set the bias of a neuron like so:
network.layers[0].neurons[0].b = -1000
print("Zero weights and negative bias:")
out = network(input, draw=True)


network.layers[0].neurons[0].b = 10000
print("Zero weights and positive bias:")
out = network(input, draw=True)

#### PUZZLES!

Below are a number of puzzles that we will solve by creating a network and assigning weights to that network. I've filled out a solution for the first puzzle and its your job to find solutions for the other three.

**Puzzle 1: Comparisons**


---



For this puzzle our network will be given a 4 dimensional input, $x$, and our goal is to create a network that returns a positive value when $(x_0 + x_1) > (x_2 + x_3)$ and return a negative value when $(x_0 + x_1) < (x_2 + x_3)$

In [ ]:
# Challenge 1: 4 dimensional comparison.

# Here we are given a 4 dimensional input. Our goal is to compare
# the sum of input[0] + input[1] against the sum of input[2] + input[3] 
# so that if input[0] + input[1] > input[2] + input[3] the result is positive
# and if input[0] + input[1] < input[2] + input[3] the result is negative.

network = NeuralNetwork(4, [1])
network.draw_net()

network.layers[0].neurons[0].w[0] = 1.0
network.layers[0].neurons[0].w[1] = 1.0
network.layers[0].neurons[0].w[2] = -1.0
network.layers[0].neurons[0].w[3] = -1.0
network.layers[0].neurons[0].b = 0

# Test our network
test_network(network, generate_comparison_data, test_comparison_result)


In [ ]:
# Challenge 2: AND circuit

# For this puzzle you will be given two dimensional input, and each
# dimension will either be -1 or +1. Your goal is to compute a logical AND.
# If both dimensions of our input are +1 then our result should be positive.
# otherwise, our result should be negative.

network = NeuralNetwork(2, [1])
network.draw_net()

network.layers[0].neurons[0].w[0] = ???
network.layers[0].neurons[0].w[1] = ???
network.layers[0].neurons[0].b = ???


# Test your network
test_network(network, generate_AND_data, test_AND_result)

In [ ]:
# Challenge 3: OR circuit

# For this puzzle you will be given two dimensional input, and each
# dimension will either be -1 or 1. Your goal is to compute a logical OR.
# If either dimensions of our input are 1 then our result should be positive.
# otherwise, our result should be negative.

network = NeuralNetwork(2, [1])
network.draw_net()

network.layers[0].neurons[0].w[0] = ???
network.layers[0].neurons[0].w[1] = ???
network.layers[0].neurons[0].b = ???


# Test your network
test_network(network, generate_OR_data, test_OR_result)

In [ ]:
# Challenge 4: XOR circuit

# For this puzzle you will be given two dimensional input, and each
# dimension will either be -1 or 1. 
# Your goal is to compute a logical XOR.
# If only 1 dimension of our input is +1 then our result should be positive.
# otherwise, our result should be negative.

# This is a problem that can (provably) not be solved with only 1 layer.
# We'll need a beefier network.

network = NeuralNetwork(2, [2, 1])
network.draw_net()

# Layer 0, neuron 0
network.layers[0].neurons[0].w[0] = ???
network.layers[0].neurons[0].w[1] = ???
network.layers[0].neurons[0].b = ???

# Layer 0, neuron 1
network.layers[0].neurons[1].w[0] = ???
network.layers[0].neurons[1].w[1] = ???
network.layers[0].neurons[1].b = ???

# Layer 1, neuron 0
network.layers[1].neurons[0].w[0] = ???
network.layers[1].neurons[0].w[1] = ???
network.layers[1].neurons[0].b = ???


# Test your network
test_network(network, generate_XOR_data, test_XOR_result)